In [3]:
import numpy as np
import math
from fractions import Fraction
import itertools
from biotuner_utils import *
from biotuner_offline import *
import matplotlib.pyplot as plt
from numpy import array, zeros, ones, arange, log2, sqrt, diff, concatenate
import emd
from PyEMD import EMD, EEMD
from scipy.signal import butter, lfilter
import colorednoise as cn
from biotuner import *
import mne

## Load dataset

In [2]:

#path = 'D:/Science/EEG_data/'
path = 'C:/Users/Dell/GitHub/CoCoBrainChannel/'
epochs = mne.read_epochs(path+'pareidolia_run2.fif')
#epochs = mne.read_epochs('C:/Users/Antoine/github/Data_EEG/pareidolia_run1.fif')
epochs = epochs.apply_baseline((-1.5, -0.1))
#epochs = epochs.crop(0.5, 7.5)
epochs_data = epochs.get_data()

Reading C:/Users/Dell/GitHub/CoCoBrainChannel/pareidolia_run2.fif ...


<ipython-input-2-48a952040299>:4: RuntimeWarning: This filename (C:/Users/Dell/GitHub/CoCoBrainChannel/pareidolia_run2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(path+'pareidolia_run2.fif')


    Found the data of interest:
        t =   -1500.00 ...    8000.00 ms
        0 CTF compensation matrices available


C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py:427: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  ch_name = ch_name[:np.argmax(ch_name == b'')].tostring()
C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py:427: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  ch_name = ch_name[:np.argmax(ch_name == b'')].tostring()
C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py:427: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  ch_name = ch_name[:np.argmax(ch_name == b'')].tostring()
C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py:427: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  ch_name = ch_name[:np.argmax(ch_name == b'')].tostring()
C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py:427: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  ch_name = ch_name[:np.argmax(ch_name == b'')].tostring()
C:\Users\Dell\Anaconda3\lib\site-packages\mne\io\tag.py

104 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Applying baseline correction (mode: mean)


## Initialize biotuner object and methods

In [19]:
class biotuner(object):
    
    '''Class used to derive peaks information, musical scales and related metrics from time series  
    
    Example of use:       
    biotuning = biotuner(sf = 1000)
    biotuning.peaks_extraction(data)
    biotuning.peaks_extension()
    biotuning.peaks_metrics()
    '''
    
    def __init__(self, sf, peaks_function = 'EEMD', precision = 0.2, compute_sub_ratios = False, 
                 n_harm = 10, harm_function = 'mult', extension_method = 'consonant_harmonic_fit'):
        '''Initializing sampling frequency'''
        self.sf = sf
        '''Initializing arguments for peak extraction
           peaks_function: method used to extract the peaks ['EEMD', 'EMD', 'HH1D_max', 'adapt', 'fixed']
           precision: precision of the peaks in Hz
           compute_sub_ratios: when set to True, include ratios < 1 in peaks_ratios attribute [True, False]'''
        self.peaks_function = peaks_function
        self.precision = precision
        self.compute_sub_ratios = compute_sub_ratios
        '''Initializing arguments for peaks metrics
           n_harm: number of harmonics to compute in harmonic_fit function
           harm_function: compute harmonics from iterative multiplication (x, 2x, 3x, ...nx) or division (x, x/2, x/3, ...x/n) ['mult', 'div']
           extension_method: ['harmonic_fit', 'consonant', 'multi_consonant', 'consonant_harmonic_fit', 'multi_consonant_harmonic_fit']'''
        self.n_harm = n_harm
        self.harm_function = harm_function
        self.extension_method = extension_method
        '''Initializing dictionary for scales metrics'''
        self.scale_metrics = {}
    
    '''First method to use. Requires data as input argument
       Generates self.peaks and self.peaks_ratios attributes'''
    
    def peaks_extraction (self, data, peaks_function = None, FREQ_BANDS = None, precision = None, sf = None, max_freq = 80, compute_sub_ratios = None):
        if sf == None:
            sf = self.sf
        if precision == None:
            precision = self.precision
        if peaks_function == None:
            peaks_function = self.peaks_function
        if compute_sub_ratios == None:
            compute_sub_ratios = self.compute_sub_ratios
        self.peaks, self.amps = compute_peaks_ts (data, peaks_function = peaks_function, FREQ_BANDS = None, precision = precision, sf = sf, max_freq = max_freq)

        a, self.peaks_ratios = compute_peak_ratios(self.peaks, rebound = True, octave = 2, sub = compute_sub_ratios)
        
    '''Generates self.extended_peaks and self.extended_peaks_ratios attributes'''
    
    def peaks_extension (self, peaks = None, n_harm = None, method = None, harm_function = 'mult', cons_limit = 0.1):
        if peaks == None:
            peaks = self.peaks
        if n_harm == None:
            n_harm = self.n_harm
        if method == None:
            method = self.extension_method
        if method == 'harmonic_fit':
            extended_peaks = harmonic_fit(peaks, self.n_harm, function = harm_function)
            self.extended_peaks = np.sort(list(self.peaks)+list(extended_peaks))
        if method == 'consonant':
            consonance, cons_pairs, cons_peaks, cons_metric = consonance_peaks (peaks, limit = cons_limit)
            self.extended_peaks = np.sort(np.round(cons_peaks, 3))
        if method == 'multi_consonant':
            consonance, cons_pairs, cons_peaks, cons_metric = consonance_peaks (peaks, limit = cons_limit)
            self.extended_peaks = np.sort(np.round(multi_consonance(cons_pairs, n_freqs = 10), 3))
        if method == 'consonant_harmonic_fit':
            extended_peaks = harmonic_fit(peaks, self.n_harm, function = harm_function)
            consonance, cons_pairs, cons_peaks, cons_metric = consonance_peaks (extended_peaks, limit = cons_limit)
            self.extended_peaks = np.sort(np.round(cons_peaks, 3))
        if method == 'multi_consonant_harmonic_fit':
            extended_peaks = harmonic_fit(peaks, self.n_harm, function = harm_function)
            consonance, cons_pairs, cons_peaks, cons_metric = consonance_peaks (extended_peaks, limit = cons_limit)
            self.extended_peaks = np.sort(np.round(multi_consonance(cons_pairs, n_freqs = 10), 3))
        self.extended_peaks_ratios = compute_peak_ratios(self.extended_peaks)[1]
      
    def compute_peaks_metrics (self, n_harm = None):
        if n_harm == None:
            n_harm = self.n_harm
            
        peaks = list(self.peaks)
        metrics = {'cons' : 0, 'euler' : 0, 'tenney': 0, 'harm_fit': 0}   
        metrics['harm_fit'] = len(harmonic_fit(peaks, n_harm = n_harm))
        a, b, c, metrics['cons'] = consonance_peaks (peaks, 0.1)
        peaks_euler = [int(round(num, 2)*1000) for num in peaks]
        metrics['euler'] = euler(*peaks_euler)
        metrics['tenney'] = tenneyHeight(peaks)
        metrics_list = []
        for value in metrics.values():
            metrics_list.append(value)
        self.peaks_metrics_list = metrics_list
        self.peaks_metrics = metrics

    '''Methods to compute scales from whether peaks or extended peaks'''
    
    def compute_diss_curve (self, input_type = 'peaks', denom=1000, max_ratio=2, consonance = True, method = 'min', plot = False):
        if input_type == 'peaks':
            peaks = self.peaks
            amps = self.amps
        if input_type == 'extended_peaks':
            peaks = self.extended_peaks
            amps = self.extended_amps

        peaks = [p*128 for p in peaks]
        amps = np.interp(self.amps, (np.array(self.amps).min(), np.array(self.amps).max()), (0.2, 0.8))
        intervals, self.diss_scale, euler_diss, diss, harm_sim_diss = diss_curve (peaks, amps, denom=denom, max_ratio=max_ratio, consonance = consonance, method = method, plot = plot)
        self.scale_metrics['diss_euler'] = euler_diss
        self.scale_metrics['dissonance'] = diss
        self.scale_metrics['diss_harm_sim'] = np.average(harm_sim_diss)
        self.scale_metrics['diss_n_steps'] = len(self.diss_scale)
        
    def compute_harmonic_entropy(self, input_type = 'peaks', res = 0.001, spread = 0.01, plot_entropy = True, plot_tenney = False):
        if input_type == 'peaks':
            ratios = self.peaks_ratios
        if input_type == 'extended_peaks':
            ratios = self.extended_peaks_ratios
        HE_scale, HE = harmonic_entropy(ratios, res = res, spread = spread, plot_entropy = plot_entropy, plot_tenney = plot_tenney)
        self.HE_scale = HE_scale[0]
        self.scale_metrics['HE'] = HE
        self.scale_metrics['HE_n_steps'] = len(self.HE_scale)  
    
    '''Generic method to fit all Biotuner methods'''
    
    def fit_all(self, data, compute_diss = True, compute_HE = True, compute_peaks_extension = True):
        biotuning = biotuner(self.sf, peaks_function = self.peaks_function, precision = self.precision, n_harm = self.n_harm)
        biotuning.peaks_extraction(data)
        biotuning.compute_peaks_metrics()
        if compute_diss == True:
            biotuning.compute_diss_curve(input_type = 'peaks', plot = False)
        if compute_peaks_extension == True:
            biotuning.peaks_extension(method = 'multi_consonant_harmonic_fit', harm_function = 'mult', cons_limit = 0.01)
        if compute_HE == True:
            biotuning.compute_harmonic_entropy(input_type = 'extended_peaks', plot_entropy = False)
        return biotuning
    
    def info(self, metrics=False, scales=False, whatever=False):
        if metrics == True:
            print('METRICS')
            print(vars(self))
        
        else:
            print(vars(self))
        return

In [16]:

data = epochs_data[10][100] # Define data (single time series)
biotuning = biotuner(1000, peaks_function = 'adapt', precision = 0.1, n_harm = 30) # Initialize biotuner object
biotuning.peaks_extraction(data)
biotuning.compute_peaks_metrics()
biotuning.compute_diss_curve(plot = False)
biotuning.peaks_extension(method = 'multi_consonant_harmonic_fit', harm_function = 'mult', cons_limit = 0.01)
biotuning.compute_harmonic_entropy(input_type = 'extended_peaks', plot_entropy = False)

In [17]:
biotuning = biotuner(1000, peaks_function = 'adapt', precision = 0.1, n_harm = 30)
biotuning = biotuning.fit_all(data)
biotuning.peaks_ratios

[1.0203488372093024,
 1.052325581395349,
 1.1279069767441863,
 1.6538461538461537,
 1.6875,
 1.7403846153846154,
 1.8092783505154637,
 1.8653846153846154,
 1.865979381443299,
 1.9392265193370164]

In [6]:
vars(biotuning)
#biotuning.HE_scale

{'sf': 1000,
 'peaks_function': 'adapt',
 'precision': 0.1,
 'compute_sub_ratios': False,
 'n_harm': 30,
 'scale_metrics': {'diss_euler': 154.0,
  'dissonance': 0.6618647437524718,
  'diss_harm_sim': 1.3188465851907345,
  'diss_n_steps': 5,
  'HE': 1.7629069799044343,
  'HE_n_steps': 15},
 'peaks': array([ 2.6,  4.3,  9.7, 18.1, 35.1]),
 'amps': array([-109.70765023, -112.02264608, -110.6194423 , -111.22711647,
        -114.06633254]),
 'peaks_ratios': [1.0203488372093024,
  1.052325581395349,
  1.1279069767441863,
  1.6538461538461537,
  1.6875,
  1.7403846153846154,
  1.8092783505154637,
  1.8653846153846154,
  1.865979381443299,
  1.9392265193370164],
 'peaks_metrics_list': [0.023439198356483965, 338, 12.332628225339015, 40],
 'peaks_metrics': {'cons': 0.023439198356483965,
  'euler': 338,
  'tenney': 12.332628225339015,
  'harm_fit': 40},
 'diss_scale': [1.353, 1.653, 1.865, 1.938, 2.0],
 'extended_peaks': array([  4.75,  10.05,  21.15,  29.6 ,  34.1 ,  54.45,  60.  ,  64.75,
     